In [1]:
from scipy.stats import chi2
import numpy as np

---
# Ejercicio 1
De acuerdo con la teoría genética de *Mendel*, cierta planta de guisantes debe producir flores blancas, rosas o rojas con probabilidad $\frac{1}{4}$, $\frac{1}{2}$ y $\frac{1}{4}$, respectivamente. Para verificar experimentalmente la teoría, se estudió una muestra de $564$ guisantes, donde se encontró que $141$ produjeron flores blancas, $291$ flores rosas y $132$ flores rojas. Aproximar el p-valor de esta muestra:

- a) utilizando la prueba de Pearson con aproximación chi-cuadrada,

## Parámetros especificados
Tenemos que :
$$
p_0 = \frac{1}{4} = P(X = \text{blancas}), \quad 
p_1 = \frac{1}{2} = P(X = \text{rosas}), \quad
p_2 = \frac{1}{4} = P(X = \text{rojas})
$$

Luego las frecuancias observadas $N_i$ son:
- $n=$ 564 guistantes.
- $N_0 = $ 141 flores blancas
- $N_1 = $ 291 flores rosas
- $N_2 = $ 132 flores rojas





Para aproximar el **p-valor** de esta muestra utilizando la prueba de Pearson con aproximación *Chi-Cuadrada* debemos tener en cuenta el estadístico
$$
T = \sum_{i=0}^n \frac{(N_i-n\times p_i)²}{n\times p_i}

$$
Tal que para nuestro caso tenemos:

$$
\begin{align*}
T &= \sum_{i=0}^2 \frac{(N_i-564\times p_i)²}{564\times p_i}\\[0.3cm]
  &=    \frac{(N_0-564 \times p_0)²}{564 \times p_0} +
        \frac{(N_1-564 \times p_1)²}{564 \times p_1} +
        \frac{(N_2-564 \times p_2)²}{564 \times p_2}\\[0.3cm]
  &=    \frac{(141-564 \times \frac{1}{4})²}{564 \times \frac{1}{4}} +
        \frac{(291-564 \times \frac{1}{2})²}{564 \times \frac{1}{2}} +
        \frac{(132-564 \times \frac{1}{4})²}{564 \times \frac{1}{4}}\\[0.3cm]
  &=    0\quad + \quad \frac{27}{94} \quad + \quad \frac{27}{47}\\[0.3cm]
  &= \frac{81}{94}
\end{align*}
$$

Ahora como hemos considerado 3 términos en el estadístico $T$, entonces debemos testear con una distribución $\chi_{3-1}^2$ tal que:
$$
p_{valor} = P(\chi_{2}^2 \geq 0.8617) \approx 0.65
$$

Ahora teniendo en cuenta que para un nivel de confianza del *95%*, como $p_{valor} > \alpha$, la hipótesis nula no se rechaza pero no hay evidencias suficientes para hacerlo.

In [4]:
def Pearson_statistic(Nsamples:int, samples:list[int], probability_values:list[float]) -> float:
    statistic = 0
    for sample, prob_i in zip(samples, probability_values):
        statistic += (sample - Nsamples * prob_i) ** 2 / (Nsamples * prob_i)
    return statistic

def Pearson_test(Nsamples: int, samples:list[int], probability_values:list[float], alpha:float):
    T = Pearson_statistic(Nsamples=Nsamples, samples=samples, probability_values=probability_values)
    print(f"🧐 T estadístico:{round(T, 4)}")

    p_value = 1 - chi2.cdf(round(T, 4), df=2)

    print(f"☝️ p-valor obtenido: {round(p_value, 4)}")

    if p_value > alpha:
        print("😒 No hay evidencia suficiente para rechazar Ho")
    else:
        print(f"🔴 Se rechaza Ho con una confianza del {100 * (100-alpha)}%")

In [6]:
#Valores constantes
FLOWERS_SAMPLES = [141, 291, 132] #blancas, rosas y rojas respectiv
PROBABILITY_VALUES = [1/4, 1/2, 1/4]
NSAMPLES = 564

alpha = 1 - 0.95 # Test para el 95%

Pearson_test(Nsamples=NSAMPLES, samples=FLOWERS_SAMPLES, probability_values=PROBABILITY_VALUES, alpha=alpha)

🧐 T estadístico:0.8617
☝️ p-valor obtenido: 0.65
😒 No hay evidencia suficiente para rechazar Ho


---
b) realizando una simulación.


Ahora a partir del estadístico obtenido, denominado $t_0= 0.8617$

In [9]:
def p_value_estimation(Nsim:int, Nsamples:int, samples:list[int], probability_values:list[float],  t0:float):
    p_value = 0
    possible_frequencies = list(range(0, len(samples)))
    for _ in range(Nsim):
        #Generamos Nsamples muestras de las posibles frecuencias observadas,
        #cada una con probabilidad p_i
        X_samples = np.random.choice(a=possible_frequencies,  p=probability_values, size=Nsamples)

        N_i = np.bincount(X_samples, minlength=len(samples))
        
        T = Pearson_statistic(Nsamples=Nsamples, samples=N_i, probability_values=probability_values)

        if T > t0:
            p_value += 1

    return p_value / Nsim

In [10]:
NSIM = 10_000
T = 0.8617
p_value = p_value_estimation(
            Nsim=NSIM,
            Nsamples=NSAMPLES,
            samples=FLOWERS_SAMPLES,
            probability_values=PROBABILITY_VALUES,
            t0=T)

print(f"☝️ p-valor obtenido: {round(p_value, 4)}")

☝️ p-valor obtenido: 0.6565
